In [64]:
import xarray
from distributed import Client


client = Client('localhost:8786')
client.restart()

<Client: 'tcp://146.6.192.149:8786' processes=0 threads=0, memory=0 B>

In [74]:
threshold_ds = xarray.Dataset(
        data_vars=dict(
            threshold=(["lat", "lon", "day", "percentile"], annual_threshold.data),
        ),
        coords=dict(
            lon=(["lon"], annual_threshold.lon.values),
            lat=(["lat"], annual_threshold.lat.values),
            day=np.arange(0, window_samples.shape[0]),
            percentile=percs
        ),
        attrs={
            "description": f"Percentile temperatures.",
            "percentiles": str(percs)
        },
    )
threshold_ds = threshold_ds.compute()

In [86]:
from numba import njit, prange,  guvectorize, int64, float32, boolean
import numba as nb
import numpy as np


@njit
def index_heatwaves(hot_days_ts: np.ndarray, max_break: int=1, min_duration: int=3) -> np.ndarray:
    """
    Identifies the heatwaves in the timeseries using the specified heatwave definition

    Keyword arguments:
    timeseries -- integer array of ones and zeros where ones indicates a hot day (numpy.ndarray)
    max_break -- the maximum number of days between hot days within one heatwave event (default 1)
    min_duration -- the minimum number of hot days to constitute a heatwave event, including after breaks (default 3)
    """
    timeseries = np.zeros(hot_days_ts.shape[0] + 2, dtype=nb.int64)
    timeseries[1:timeseries.shape[0]-1] = hot_days_ts

    diff_indices = np.where(np.diff(timeseries) != 0)[0] + 1

    in_heatwave = False
    current_hw_index = 1

    hw_indices = np.zeros(timeseries.shape, dtype=nb.int64)

    broken = False
    for i in range(diff_indices.shape[0]-1):
        index = diff_indices[i]
        next_index = diff_indices[i+1]

        if timeseries[index] == 1 and in_heatwave:
            hw_indices[index:next_index] = current_hw_index
        elif timeseries[index] == 0 and in_heatwave and next_index-index <= max_break and not broken:
            hw_indices[index:next_index] = current_hw_index
            broken = True
        elif timeseries[index] == 1 and not in_heatwave and next_index-index >= min_duration:
            in_heatwave = True
            hw_indices[index:next_index] = current_hw_index
        elif in_heatwave:
            current_hw_index += 1
            in_heatwave = False
            broken = False
    return timeseries[1:-1]*hw_indices[1:-1]

@njit
def heatwave_frequency(hw_ts: np.array) -> int:
    return np.sum(hw_ts > 0)

@njit
def heatwave_duration(hw_ts: np.array) -> int:
    hwd = 0
    for value in np.unique(hw_ts):
        index_count = 0
        if value != 0:
            for day in hw_ts:
                if day == value:
                    index_count += 1
        if index_count > hwd:
            hwd = index_count
    return hwd


def get_range_indices(times: np.array, start: tuple, end: tuple):
    num_years = times[-1].year - times[0].year + 1
    ranges = np.zeros((num_years, 2), dtype=int) - 1

    n = 0
    looking_for_start = True
    for t in range(times.shape[0]):
        if looking_for_start:
            if times[t].month == start[0] and times[t].day == start[1]:
                looking_for_start = False
                ranges[n, 0] = t
        else:
            if times[t].month == end[0] and times[t].day == end[1]:
                looking_for_start = True
                ranges[n, 1] = t
                n += 1

    if not looking_for_start:
        ranges[-1, -1] = times.shape[0]

    return ranges


def compute_hemisphere_ranges(temperatures: xarray.DataArray):
    north_ranges = get_range_indices(temperatures.time.values, (5, 1), (10, 1))
    south_ranges = get_range_indices(temperatures.time.values, (10, 1), (3, 1))

    ranges = np.zeros((north_ranges.shape[0], 2, temperatures.shape[1], temperatures.shape[2]), dtype=int) - 1

    for i in range(temperatures.shape[1]):
        for j in range(temperatures.shape[2]):
            if i < ranges.shape[2] / 2:
                ranges[:, :, i, j] = south_ranges
            else:
                ranges[:, :, i, j] = north_ranges

    return ranges


def build_doy_map(temperatures: xarray.DataArray, threshold: xarray.DataArray):
    doy_map = np.zeros(temperatures.shape[0], dtype=int) - 1
    for time_index, time in enumerate(temperatures.time.values):
        doy_map[time_index] = time.dayofyr - 1
    return doy_map


@njit(parallel=True)
def compute_heatwave_metric(heat_stat_func, season_ranges: np.array, heatwave_indices: np.array):
    metric = np.zeros((season_ranges.shape[0], season_ranges.shape[2], season_ranges.shape[3]), dtype=nb.int64)
    for y in prange(metric.shape[0]):
        for i in prange(metric.shape[1]):
            for j in prange(metric.shape[2]):
                start = season_ranges[y, 0, i, j]
                end = season_ranges[y, 1, i, j]
                metric[y, i, j] = heat_stat_func(heatwave_indices[start:end, i, j])

    return metric


@njit(parallel=True)
def compute_int64_spatial_func(ts_spatial_array, func):
    results = np.zeros(ts_spatial_array.shape, nb.int64)

    for i in prange(ts_spatial_array.shape[1]):
        for j in prange(ts_spatial_array.shape[2]):
            results[:, i, j] = func(ts_spatial_array[:, i, j])
    return results


@njit(parallel=True)
def indicate_hot_days(temperatures: np.ndarray, threshold: np.ndarray, doy_map: np.ndarray) -> np.ndarray:
    hot_days = np.zeros(temperatures.shape, dtype=nb.boolean)

    for time_index in prange(temperatures.shape[0]):
        if doy_map[time_index] >= 0:
            hot_days[time_index] = temperatures[time_index] > threshold[doy_map[time_index]]
    return hot_days


#####
def compute_heatwave_metrics(future_dataset: xarray.DataArray, threshold: xarray.DataArray):
    datasets = []
    for perc in threshold.percentile.values:
        doy_map = build_doy_map(future_dataset, threshold["threshold"])
        hot_days = indicate_hot_days(future_dataset.values, threshold["threshold"].sel(percentile=perc).values, doy_map)
        heatwave_indices = compute_int64_spatial_func(hot_days, index_heatwaves)
        season_ranges = compute_hemisphere_ranges(future_dataset)

        metrics_ds = xarray.Dataset(data_vars={
                "HWF": (["year", "lat", "lon"], compute_heatwave_metric(heatwave_frequency, season_ranges, heatwave_indices)),
                "HWD": (["year", "lat", "lon"], compute_heatwave_metric(heatwave_duration, season_ranges, heatwave_indices))
            },
            coords=dict(
                year=np.arange(future_dataset.time.values[0].year, future_dataset.time.values[-1].year + 1),
                lat=future_dataset.lat.values,
                lon=future_dataset.lon.values,
                percentile=perc
            ))
        datasets.append(metrics_ds)

    dataset = xarray.concat(datasets, dim="percentile")
    dataset.attrs = {
        "dev_name" : "Cameron Cummins",
        "dev_affiliation" : "Persad Aero-Climate Lab, Department of Earth and Planetary Sciences, The University of Texas at Austin",
        "dev_email" : "cameron.cummins@utexas.edu",
        "description": "Heatwave metrics.",
        "date_prepared" : str(datetime.now())
    }

    return dataset

In [78]:
future_temps = xarray.open_zarr("/projects/dgs/persad_research/SIMULATION_DATA/ZARR/RAMIP/SIM_VARIABLES/CESM2_ssp370-126aer_day_tasmax.zarr")

In [116]:
def build_doy_map(temperatures: xarray.DataArray, threshold: xarray.DataArray):
    doy_map = np.zeros(temperatures.time.values.size, dtype=int) - 1
    for time_index, time in enumerate(temperatures.time.values):
        doy_map[time_index] = time.dayofyr - 1
    return doy_map


doy_map1 = build_doy_map(future_temps["tasmax"], threshold_ds["threshold"])
doy_map1

array([  0,   1,   2, ..., 362, 363, 364])

In [ ]:
@njit(parallel=True)
def indicate_hot_days(temperatures: np.ndarray, threshold: np.ndarray, doy_map: np.ndarray) -> np.ndarray:
    hot_days = np.zeros(temperatures.shape, dtype=nb.boolean)

    for time_index in prange(temperatures.shape[0]):
        if doy_map[time_index] >= 0:
            hot_days[time_index] = temperatures[time_index] > threshold[doy_map[time_index]]
    return hot_days


indicate_hot_days(future_temps, threshold["threshold"].sel(percentile=perc).values, doy_map)

In [119]:
type(doy_map[2])

numpy.int64

In [102]:
sample = future_temps["tasmax"].sel(member='r4i1p1f1').sel(lat=0, lon=0, method='nearest').compute()

In [147]:
@nb.guvectorize(
    [(nb.float64[:],
      nb.int64,
      nb.int64,
      nb.int64[:]
     )],
    '(t), (), () -> (t)'
)
def index_heatwaves(hot_days_ts: np.ndarray, max_break: int, min_duration: int, output: np.ndarray) -> np.ndarray:
    """
    Identifies the heatwaves in the timeseries using the specified heatwave definition

    Keyword arguments:
    timeseries -- integer array of ones and zeros where ones indicates a hot day (numpy.ndarray)
    max_break -- the maximum number of days between hot days within one heatwave event (default 1)
    min_duration -- the minimum number of hot days to constitute a heatwave event, including after breaks (default 3)
    """
    timeseries = np.zeros(hot_days_ts.shape[0] + 2, dtype=nb.int64)
    timeseries[1:timeseries.shape[0]-1] = hot_days_ts

    diff_indices = np.where(np.diff(timeseries) != 0)[0] + 1

    in_heatwave = False
    current_hw_index = 1

    hw_indices = np.zeros(timeseries.shape, dtype=nb.int64)

    broken = False
    for i in range(diff_indices.shape[0]-1):
        index = diff_indices[i]
        next_index = diff_indices[i+1]

        if timeseries[index] == 1 and in_heatwave:
            hw_indices[index:next_index] = current_hw_index
        elif timeseries[index] == 0 and in_heatwave and next_index-index <= max_break and not broken:
            hw_indices[index:next_index] = current_hw_index
            broken = True
        elif timeseries[index] == 1 and not in_heatwave and next_index-index >= min_duration:
            in_heatwave = True
            hw_indices[index:next_index] = current_hw_index
        elif in_heatwave:
            current_hw_index += 1
            in_heatwave = False
            broken = False
    output = timeseries[1:-1]*hw_indices[1:-1]


In [158]:
def get_range_indices(times: np.array, start: tuple, end: tuple):
    num_years = times[-1].year - times[0].year + 1
    ranges = np.zeros((num_years, 2), dtype=int) - 1

    n = 0
    looking_for_start = True
    for t in range(times.shape[0]):
        if looking_for_start:
            if times[t].month == start[0] and times[t].day == start[1]:
                looking_for_start = False
                ranges[n, 0] = t
        else:
            if times[t].month == end[0] and times[t].day == end[1]:
                looking_for_start = True
                ranges[n, 1] = t
                n += 1

    if not looking_for_start:
        ranges[-1, -1] = times.shape[0]

    return ranges


def compute_hemisphere_ranges(temperatures: xarray.DataArray):
    north_ranges = get_range_indices(temperatures.time.values, (5, 1), (10, 1))
    south_ranges = get_range_indices(temperatures.time.values, (10, 1), (3, 1))

    ranges = np.zeros((north_ranges.shape[0], 2, temperatures.lat.size, temperatures.lon.size), dtype=int) - 1

    for i in range(temperatures.lat.size):
        for j in range(temperatures.lon.size):
            if i < ranges.shape[2] / 2:
                ranges[:, :, i, j] = south_ranges
            else:
                ranges[:, :, i, j] = north_ranges

    return ranges

In [150]:
@nb.guvectorize(
    [(nb.int64[:],
      nb.int64
     )],
    '(t) -> ()'
)
def heatwave_frequency(hw_ts: np.ndarray, season_ranges: np.ndarray, output: np.ndarray) -> int:
    output = np.sum(hw_ts > 0)

In [159]:
season_ranges = compute_hemisphere_ranges(future_temps["tasmax"])
season_ranges.shape

(65, 2, 192, 288)

In [170]:
times[-1]

cftime.DatetimeNoLeap(2079, 12, 31, 12, 0, 0, 0, has_year_zero=True)

In [169]:
np.arange(times[0].year, times[-1].year + 1, 1)

array([2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025,
       2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036,
       2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047,
       2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058,
       2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069,
       2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079])

In [183]:
index_test.dims

('member', 'lat', 'lon', 'percentile', 'time')

In [187]:
ds = metric_test.compute()

<xarray.DataArray (member: 10, lat: 192, lon: 288, percentile: 3, year: 65)> Size: 863MB
array([[[[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],

         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],

         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],

         ...,

         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],

         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
...
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],

         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],

         ...,

         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],

         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],

         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]]]]])
Coordinates:
    height      float64 8B 2.0
  * lat         (lat) float64 2kB -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * lon         (lon) float64 2kB 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * member      (member) <U9 360B 'r4i1p1f1' 'r8i1p1f1' ... 'r3i1p1f1'
  * percentile  (percentile) float64 24B 0.9 0.91 0.92
  * year        (year) int64 520B 2015 2016 2017 2018 ... 2076 2077 2078 2079

In [188]:
metric_test

<xarray.DataArray (member: 10, lat: 192, lon: 288, percentile: 3, year: 65)> Size: 863MB
dask.array<transpose, shape=(10, 192, 288, 3, 65), dtype=int64, chunksize=(1, 96, 72, 3, 65), chunktype=numpy.ndarray>
Coordinates:
    height      float64 8B ...
  * lat         (lat) float64 2kB -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * lon         (lon) float64 2kB 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * member      (member) <U9 360B 'r4i1p1f1' 'r8i1p1f1' ... 'r3i1p1f1'
  * percentile  (percentile) float64 24B 0.9 0.91 0.92
  * year        (year) int64 520B 2015 2016 2017 2018 ... 2076 2077 2078 2079